In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import load_model
from multiprocessing import cpu_count
from warnings import catch_warnings
from warnings import filterwarnings
from pandas import read_csv
import matplotlib.pyplot as plt
from pandas import DataFrame
import time
#from datetime import datetime
import datetime
import itertools
import mysql.connector
from mysql.connector import connection
from mysql.connector import errorcode
from statistics import mean

In [2]:
def loadData(t):
    myconn = mysql.connector.connect(host="82.180.142.51",
                                     database="u978805288_land",
                                     user="u978805288_root",
                                     password="ACSL@b123")
    (myconn)

    sql_select_query = "select sensor_id, value from data_log, info_time where data_log.info_id = info_time.info_id AND data_log.triplet_id = '" + \
        t+"' ORDER BY data_log.info_id desc limit 60"
    date_select_query = "select Date_Time from info_time where triplet_id='" + \
        t+"' order by info_id desc limit 1"

    cursor = myconn.cursor()
    cursor.execute(sql_select_query)
    data = cursor.fetchall()

    cursor.execute(date_select_query)
    date = cursor.fetchall()

    myconn.close()
    return data, date[0][0]

In [3]:
# triplet_id = "t38"
# data, date_value = loadData(triplet_id)
# print("Data:")
# # print(data)
# for row in data:
#     print(row)

# print("Date:", date_value)

In [4]:
def dataPreprocess(data):
    result = []
    data.reverse()

    combined_data = []
    for i in range(0, len(data), 7):
        row = data[i:i+7]
        sensor_data = []

        for s, d in row:
            sid = s
            if sid == 's6':
                continue
            elif sid == 's5':
                x = d.split(',')[:6]
                for j in x:
                    sensor_data.append(float(j))
                x = d.split(',')[-1]
                sensor_data.append(float(x))
            elif sid.startswith('s'):
                x = d.split(',')[:]
                for j in x:
                    sensor_data.append(float(j))

        combined_data.extend(sensor_data)

    data = np.array(combined_data)
    return combined_data


# preprocessed_data = dataPreprocess(data)
# split_rows = [preprocessed_data[i:i+13] for i in range(0, len(preprocessed_data), 13)]

# print(preprocessed_data)
# for row in split_rows:
#     print(row)

In [5]:
# data_list = []
# count = 0
# for i in range(10):
#     d = preprocessed_data[count:count+13]
#     data_list.append(d)
#     count += 13

# data_array = np.array(data_list)
# data_array = data_array.reshape((-1, 10, 13))
# data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
# print(data_array)

In [6]:
STOREDIR = ('finalmodel')
model1 = tf.saved_model.load(STOREDIR)
predict_fn = model1.signatures["serving_default"]

# Load the second model
model2 = tf.keras.models.load_model('website1.h5')


def runModel(data_tensor):
    previous = 1
    try:
        # Run the prediction on the first model
        pred = predict_fn(data_tensor)
        pred = np.argmax(pred)
        pred = np.round(pred).astype(int)

        # Run the prediction on the second model
        dt = data_tensor[:,:,0:5]
        pred1 = model2.predict(dt)

        combined_list = []
        combined_list.extend([round(x,2) for x in pred1.tolist()[0]])
        combined_list.append(pred)

        return (combined_list)
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [7]:
import requests
import datetime


def insertDatabase(prediction, triplet_id):

    connection = mysql.connector.connect(host = "82.180.142.51",

                                database = "u978805288_land",

                                user =   "u978805288_root",

                                password= "ACSL@b123")

    try:
        (connection)

        # current_time = date + datetime.timedelta()
        current_time = datetime.datetime.now()  # Get the current time
        current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
        print('current_time:',current_time)


        val = "(NULL,'" +(triplet_id)+"',"+ str(prediction[0])+","+ str(prediction[1])+","+str(prediction[2])+","+str(prediction[3])+","+str(prediction[4])+","+str(prediction[5])+ ",'"+str(current_time)+ "','no')"


        add_prediction = "INSERT INTO Prediction_Real_Data  VALUES "+val
        print(add_prediction)


        cursor = connection.cursor()

        cursor.execute(add_prediction)
        connection.commit()

        print(cursor.rowcount, "Record inserted successfully into prediction table")
        cursor.close()


    except mysql.connector.Error as error:

        print("Failed to insert record into prediction table {}".format(error))


    finally:

        if connection.is_connected():
            connection.close()

In [11]:
"""
correct: t33, t28, t38, t39, t55, t61, t76
same data by model: t36 t37 t40 t41 t51 t63 t64 t70 t71 t74 t75 t77 t82 t83
error: t50 t53 t67 t69 t72 t73 t78 t79 t81 t84 t29 t32 t54 t56 t80
"""


station = ['t27','t28','t36','t37','t38','t39','t40','t41','t50','t51','t52','t53','t54','t55','t56','t61','t62','t63','t64','t65','t66','t67','t68','t69','t70','t71','t72','t73','t74','t75','t78','t79','t80','t81','t82','t83','t84']

location_id = {
    "t27": {"lat": 31.923, "lon": 77.236},  # balianala
    "t32": {"lat": 30.888, "lon": 77.057},  # Kumarhatti
    "t36": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak2
    "t37": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak1
    "t38": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak3
    "t40":  {"lat": 31.351, "lon": 77.618},  # Ghoda farm 1
    "t41":  {"lat": 31.351, "lon": 77.618},  # Ghoda farm 2
    "t50": {"lat": 31.781, "lon": 76.994},  # IIT Mandi A9
    "t51": {"lat": 31.403, "lon": 78.303},  # Batseri kinnaur
    "t52": {"lat": 31.554, "lon": 77.810},  # nigulsari kinnaur
    "t53": {"lat": 31.534, "lon": 78.130},  # Urni Dhank kinnaur
    "t54": {"lat": 31.532, "lon": 78.047},  # Pagal_Nala_Kinnaur
    "t55": {"lat": 31.433, "lon": 78.247},  # Kuppa_Kinnaur
    "t56": {"lat": 31.590, "lon": 78.300},  # purbani_Kinnaur
    "t61": {"lat": 31.248, "lon": 77.089},  # tattapni_mandi
    "t62": {"lat": 31.400, "lon": 77.206},  # sanarli_1
    "t63": {"lat": 31.923, "lon": 77.236},  # sanarli_2
    "t64": {"lat": 31.923, "lon": 77.236},  # sanarli_3
    "t65": {"lat": 31.923, "lon": 77.236},  # Garud_nala
    "t67": {"lat": 31.777, "lon": 76.971},  # Griffon Peak IIT mandi
    "t69": {"lat": 31.351, "lon": 77.618},  # Ghoda farm 4
    "t70": {"lat": 31.351, "lon": 77.618},  # Ghoda farm 5
    "t71": {"lat": 31.851, "lon": 76.635},  # sandhol
    "t72": {"lat": 32.226, "lon": 76.326},  # dharmshala
    "t73": {"lat": 32.242, "lon": 76.321},  # mcleaodganj
    "t74": {"lat": 31.789, "lon": 76.267},  # chola kangra
    "t75": {"lat": 31.917, "lon": 76.120},  # Diba shahpur
    "t78": {"lat": 32.101, "lon": 76.273},  # Kangra colony
    "t79": {"lat": 32.292, "lon": 75.996},  # Bariara
    "t80": {"lat": 32.311, "lon": 75.895},  # Baldun
    "t81": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak
    "t82": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak
    "t83": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak9
    "t84": {"lat": 31.777, "lon": 76.971},  # Griffon_Peak
}

In [9]:
def fetch_weather_data(location, locations):
    if location in locations:
        lat = locations[location]['lat']
        lon = locations[location]['lon']
        api_url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid=bdc503040a4bbf2de4eeb9eb3ab841ed&units=metric"
        response = requests.get(api_url)
        data = response.json()
        forecasts = []

        # Get current time
        current_time = datetime.datetime.now()
        formatted_time = current_time.strftime('%H:%M:%S')

        # Find the next future forecast with a 3-hour interval
        for forecast in data['list']:
            forecast_time = datetime.datetime.strptime(
                forecast['dt_txt'], '%Y-%m-%d %H:%M:%S')
            if forecast_time > current_time:
                # Convert forecast time to local timezone
                forecast_time = forecast_time.astimezone()

                # If the current time is after 21:00, fetch the details for the next day at 00:00
                if current_time.hour >= 21:
                    if forecast_time.hour == 0:
                        date_time = forecast_time.strftime('%Y-%m-%d %H:%M:%S')
                        print(date_time)
                        temperature = forecast['main']['temp']
                        humidity = forecast['main']['humidity']
                        pressure = forecast['main']['pressure']
                        rain = forecast['rain']['3h'] if 'rain' in forecast and '3h' in forecast['rain'] else 0
                        forecasts.append(
                            [temperature, humidity, pressure, rain, 0, 0])
                        break  # Stop iterating once the next future forecast is found
                else:
                    # Ensure the forecast time is in the future and has a 3-hour interval
                    if forecast_time.hour % 3 == 0:
                        date_time = forecast_time.strftime('%Y-%m-%d %H:%M:%S')
                        temperature = forecast['main']['temp']
                        humidity = forecast['main']['humidity']
                        pressure = forecast['main']['pressure']
                        rain = forecast['rain']['3h'] if 'rain' in forecast and '3h' in forecast['rain'] else 0
                        forecasts.append(
                            [temperature, humidity, pressure, rain, 0, 0])
                        break  # Stop iterating once the next future forecast is found

        return forecasts
    else:
        return None

In [18]:

# Iterate over each location and fetch weather data using API
for location, coords in location_id.items():
    # Pass the locations dictionary as an argument
    forecast = fetch_weather_data(location, location_id)
    if forecast:
        print(f"Weather forecast for {location}:")
        print(forecast[0])
    else:
        print(f"Failed to fetch weather data for {location}.")

Weather forecast for t27:
[4.07, 67, 1016, 0, 0, 0]
Weather forecast for t32:
[16.16, 46, 1008, 0, 0, 0]
Weather forecast for t38:
[12.54, 65, 1012, 0, 0, 0]
Weather forecast for t50:
[15.09, 66, 1013, 0, 0, 0]
Weather forecast for t52:
[11.41, 77, 1019, 0, 0, 0]
Weather forecast for t53:
[7.28, 76, 1019, 0, 0, 0]
Weather forecast for t54:
[12.07, 77, 1019, 0, 0, 0]
Weather forecast for t55:
[6.8, 72, 1019, 0, 0, 0]
Weather forecast for t56:
[8.44, 73, 1020, 0, 0, 0]
Weather forecast for t61:
[17.42, 68, 1012, 0, 0, 0]
Weather forecast for t62:
[12.02, 75, 1014, 0, 0, 0]
Weather forecast for t65:
[4.07, 67, 1016, 0, 0, 0]
Weather forecast for t67:
[12.54, 65, 1012, 0, 0, 0]


In [10]:

while True:
    count = 0
    for triplet_id in station:
        data, date = loadData(triplet_id)
        # print("yo: ",date)
        date = date + datetime.timedelta(10)
        # current_time = datetime.datetime.now()  # Get the current time
        if triplet_id in location_id:
            # Fetch weather data using the lat long from location_id
            location = triplet_id
            forecast = fetch_weather_data(location, location_id)
            if forecast:
                print(f"Weather forecast for {location}:")
                print(forecast[0])
                insertDatabase(forecast[0], triplet_id)
            else:
                print(f"Failed to fetch weather data for {location}.")
        else:
            preprocessed_data = dataPreprocess(data)
            data_list = []
            count = 0
            for i in range(10):
                d = preprocessed_data[count:count+13]
                data_list.append(d)
                count += 13

            data_array = np.array(data_list)
            data_array = data_array.reshape((-1, 10, 13))
            data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
            prediction = runModel(data_tensor)
            print(prediction)
            insertDatabase(prediction, triplet_id)

        print('Enjoy')

    count += 1
    time.sleep(10)

Weather forecast for t27:
[2.6, 58, 1015, 0, 0, 0]
current_time: 2024-04-23 18:16:45
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t27',2.6,58,1015,0,0,0,'2024-04-23 18:16:45','no')
1 Record inserted successfully into prediction table
Enjoy
1/1 [==============================] - 1s 799ms/step
[17.7, 71.58, 924.44, 0.0, 0.0, 0]
current_time: 2024-04-23 18:16:59
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t28',17.7,71.58,924.44,0.0,0.0,0,'2024-04-23 18:16:59','no')
1 Record inserted successfully into prediction table
Enjoy
Weather forecast for t36:
[11.37, 53, 1011, 0, 0, 0]
current_time: 2024-04-23 18:17:02
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t36',11.37,53,1011,0,0,0,'2024-04-23 18:17:02','no')
1 Record inserted successfully into prediction table
Enjoy
Weather forecast for t37:
[11.37, 53, 1011, 0, 0, 0]
current_time: 2024-04-23 18:17:09
INSERT INTO Prediction_Real_Data  VALUES (NULL,'t37',11.37,53,1011,0,0,0,'2024-04-23 18:17:09','no')
1 Record inserted successfully i

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (10,) + inhomogeneous part.